In [27]:
import os
import importlib
import torch

from voc12 import my_dataloader
from torch.utils.data import DataLoader
from utility.image_util import *
from tqdm import tqdm
from utility import image_util

from misc import imutils
import torch.nn.functional as F

In [28]:
cam_network = "net.resnet50_cam"
cam_root = "../irn_result/cam/"
rgbd_cam_root = "../irn_result/rgbd_cam"
val_list = "voc12/val.txt"
voc12_root = "../Dataset/VOC2012/"
depth_root = "../result/depth_img/"

device = "cuda"
rgbd_cam_weights_name = "../sess/voc_sess/resnet50_rgbd_cam.pth"
cam_weights_name = "../sess/voc_sess/resnet50_cam.pth"
scales = (1.0, 0.5, 1.5, 2.0)

In [29]:
model = getattr(importlib.import_module("net.resnet50_cam"), 'Net')(rgbd=True)

# model.load_state_dict(torch.load(cam_weights_name), strict=True)
model.load_state_dict(torch.load(rgbd_cam_weights_name), strict=True)

model.eval()
model = model.to(device)

In [30]:
# dataset = my_dataloader.VOC12ClassificationDataset(val_list, voc12_root= voc12_root)
dataset = my_dataloader.VOC12_DepthClassificationDataset(val_list, voc12_root= voc12_root, depth_root=depth_root)


data_loader = DataLoader(dataset)

In [36]:
print(dataset[15]['img'][3])

[[ 3.0720666e-41  3.0720666e-41  8.0077699e-04 ...  1.8155022e-02
  -1.9448768e-02 -7.3257852e-03]
 [ 5.8058789e-04  1.7402628e-02  8.6773623e-05 ...  1.1792872e-02
   1.2564420e-02  1.6304037e-02]
 [ 2.8246477e-02  2.1779416e-02  1.7464655e-02 ...  1.3903867e-02
   1.3176864e-02 -1.4397801e-02]
 ...
 [-5.3211331e-01 -7.9355121e-01 -8.1098038e-01 ... -7.0640522e-01
   1.4548148e+00 -6.0183007e-01]
 [ 4.4392157e-01 -4.4496733e-01 -3.7525055e-01 ...  7.7908494e-02
  -1.1381263e-01 -7.7612203e-01]
 [-5.4954249e-01  1.4762527e-01 -1.0724183e+00 ... -1.6475817e+00
   1.4762527e-01 -8.4583879e-01]]


In [ ]:
with torch.no_grad():
    for iter, pack in enumerate(tqdm(data_loader)):
        img = pack['img'].to(device)
        x = model(img)
        print(x)